## An example of crawling one page of Pantip.com
Reference:

Scrollintoview https://docs.w3cub.com/dom/element/scrollintoview/

WebDriverWait https://www.seleniumhq.org/docs/04_webdriver_advanced.jsp

In [1]:
from selenium import webdriver
# from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re

### Assign some arguments

In [2]:
url = 'https://pantip.com/topic/38478051'

options = webdriver.ChromeOptions()
#options.add_argument('headless')

# initialize the driver
driver = webdriver.Chrome(executable_path="C:\webdriver\chromedriver.exe", options=options)
driver.get(url)

### Click button to accept cookies
Reference https://selenium.dev/selenium/docs/api/py/webdriver_support/selenium.webdriver.support.expected_conditions.html

![](img\cookies.png)


In [3]:
try:
    cookie = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pt-snackbar__actions.pt-action-policy")))
    cookie.click()
except TimeoutException as ex:
    print("There is not the pop up")

### Click to load page
Reference https://selenium-python.readthedocs.io/api.html

![](img\load_page.png)

In [4]:
loadPage = True
while loadPage:
    try:        
        load_comment_buttons = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.bar-paging-ed span.focus-txt")))
        driver.execute_script("return arguments[0].scrollIntoView({block: \"center\", inline: \"center\"});", load_comment_buttons)
        load_comment_buttons.click()
    except TimeoutException as ex:
        loadPage = False

### Click reply buttons

![](img\reply_button.png)

In [5]:
try:
    comment_buttons = WebDriverWait(driver, 2).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "a.reply.see-more span.focus-txt")))    
    for button in comment_buttons:
        driver.execute_script("return arguments[0].scrollIntoView({block: \"center\", inline: \"center\"});", button)
        button.click()
except TimeoutException as ex:
    print("No reply")

###  Loop click sub reply buttons
![](img\sub_reply_button.png)


In [6]:
loadButton = True   
while loadButton:
    try:
        load_comment_buttons = WebDriverWait(driver, 2).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "a.load-reply-next span.focus-txt"))) 
        for button in load_comment_buttons:
            driver.execute_script("return arguments[0].scrollIntoView({block: \"center\", inline: \"center\"});", button)
            button.click()
    except TimeoutException as ex:
        loadButton = False

### Get full HTML

In [7]:
content = driver.page_source
soup = BeautifulSoup(content,'lxml')
driver.close()

### Get the owner's post

In [8]:
 # Extract main post informaiton
pantip = dict()
all_comment = list()

main_area = soup.find('div', class_ = 'main-post-inner sticky-navi-comment')

# Get rid of edit-history
edit_history = main_area.find('div', class_ = 'edit-history')
if edit_history is not None:
    edit_history.decompose()

pantip['title'] = main_area.find(class_ = 'display-post-title').get_text().strip()
pantip['url'] =  url
main = {
        'post' : main_area.find(class_ = 'display-post-story').get_text().strip(),
        'post_number' : 'ความคิดเห็นที่ 0',
        'owner' : main_area.find(class_ = 'display-post-name owner').get_text().strip(),
        'time' : main_area.find(class_ = 'timeago').get_text().strip()
        }
all_comment.append(main)
all_comment[0]

{'post': 'เพื่อนๆ คนไหนเคยทำ ช่วยแชร์ประสบการณ์หน่อยนะครับ',
 'post_number': 'ความคิดเห็นที่ 0',
 'owner': 'ทนายขีดเขียน',
 'time': '20 มกราคม 2562 เวลา 23:24 น.'}

### Loop get all posts

In [12]:
# Get all replies
for div in soup.find_all("div", id = re.compile("^(comment|reply)-[0-9]+")):

    # Get rid of edit-history
    edit_history = div.find('div', class_ = 'edit-history')
    if edit_history is not None:
        edit_history.decompose()

    # Extract information
    post = div.find(class_ = 'display-post-story').get_text().strip()
    if post.find('ความคิดเห็นนี้ถูกลบเนื่องจาก') == -1:
        reply ={
                'post' : post,
                'post_number' : div.find(class_ = 'display-post-number').get_text().strip(),
                'post_owner' : div.find(class_ = 'display-post-name').get_text().strip(),
                'time' : div.find(class_ = 'timeago').get_text().strip()
                }
        all_comment.append(reply)
pantip['sub_post'] = all_comment

### Data structure

In [14]:
pantip.keys()

dict_keys(['title', 'url', 'sub_post'])

In [15]:
pantip['title']

'อายุ 35 เงินเดือน 100,000 บาท อาศัยอยู่ใน กทม. เพื่อนๆ คิดว่า เพียงพอที่จะให้แฟนลาออกจากงานมาดูลูกอย่างเดียวไหมครับ'

In [16]:
pantip['url']

'https://pantip.com/topic/38478051'

In [18]:
pantip['sub_post'][:3]

[{'post': 'เพื่อนๆ คนไหนเคยทำ ช่วยแชร์ประสบการณ์หน่อยนะครับ',
  'post_number': 'ความคิดเห็นที่ 0',
  'owner': 'ทนายขีดเขียน',
  'time': '20 มกราคม 2562 เวลา 23:24 น.'},
 {'post': 'ผญ.บางคนชอบทำงาน อยู่บ้านไม่มีสังคมจะเบื่อ ถามแฟนดีที่สุดครับ',
  'post_number': 'ความคิดเห็นที่ 1',
  'post_owner': 'สมาชิกหมายเลข 4054065',
  'time': '20 มกราคม 2562 เวลา 23:28 น.'},
 {'post': 'เงินเดือนขนาดนี้ ลาออกเลี้ยงลูกสบาย',
  'post_number': 'ความคิดเห็นที่ 2',
  'post_owner': 'สมาชิกหมายเลข 4158576',
  'time': '20 มกราคม 2562 เวลา 23:29 น.'}]